In [1]:
import deimos_router

In [22]:
from deimos_router import Router, register_router, chat
from deimos_router.rules import TaskRule, AutoTaskRule, CodeRule, CodeLanguageRule, NaturalLanguageRule

## Basic Usage - Task Based Routing

A Router uses one or more Rules to select a model. The simplest Rule is TaskRule, which maps task names to models.

In [3]:
Router(
   name="my-first-router",
   rules=[
       TaskRule(
           name="task-based-routing",
           triggers={
               'coding': 'openai/gpt-5',
               'creative': 'openai/gpt-4o',
               'simple': 'openai/gpt-5-nano'
           }
       )
   ],
   default="openai/gpt-4o-mini"
)



Router(name='my-first-router', models=[])

To use a Router, make a call to `chat.completions.create`, as if it were a call to OpenAI's SDK, and specify the router as the model.

For simple task-based routing, include the task name as an argument.

In [4]:
# Use the router for chat completions
response = chat.completions.create(
   model="deimos/my-first-router",
   messages=[
       {"role": "user", "content": "Write a python function that finds the nth fibonacci number"}
   ],
   task="coding"
)

print(response.choices[0].message.content)

def fib(n: int) -> int:
    """
    Return the nth Fibonacci number F_n with F_0 = 0, F_1 = 1.
    Uses fast doubling for O(log n) time and O(log n) space.

    Args:
        n: non-negative integer index

    Returns:
        F_n as an int

    Raises:
        TypeError: if n is not an int
        ValueError: if n is negative
    """
    if not isinstance(n, int):
        raise TypeError("n must be a non-negative integer")
    if n < 0:
        raise ValueError("n must be non-negative")

    def _dob(k: int) -> tuple[int, int]:
        # Returns (F_k, F_{k+1})
        if k == 0:
            return 0, 1
        a, b = _dob(k >> 1)         # a=F_m, b=F_{m+1}, where m = k//2
        c = a * ((b << 1) - a)      # F_{2m}
        d = a * a + b * b           # F_{2m+1}
        if k & 1:
            return d, c + d         # (F_{2m+1}, F_{2m+2})
        else:
            return c, d             # (F_{2m}, F_{2m+1})

    return _dob(n)[0]


Details about the routing decision can be found in the response at `._deimos_metadata`.

In [5]:
response._deimos_metadata

{'router_used': 'my-first-router',
 'selected_model': 'openai/gpt-5',
 'original_model_field': 'openai/gpt-5',
 'available_models': [],
 'explain': [{'rule_type': 'TaskRule',
   'rule_name': 'task-based-routing',
   'rule_trigger': 'coding',
   'decision': 'openai/gpt-5'}]}

## AutoTask Routing

An `AutoTaskRule` is created in the same way as a `TaskRule`, but the task is determined by a call to a small language model.

In [6]:
auto_task = AutoTaskRule(
    name = "auto-task-rule",
    triggers = {
        "creative writing" : "openai/gpt-4o",
        "writing code" : "openai/gpt-5",
        "informational" : "openai/gpt-5-mini",
        "haiku composition" : "openai/gpt-5-nano"
    },
)


        

In [7]:
auto_router = Router(
    name = "auto-router",
    rules = [auto_task],
    default="openai/gpt-4o-mini"
    )


In [8]:
# Use the router for chat completions
response1 = chat.completions.create(
   model="deimos/auto-router",
   messages=[
       {"role": "user", "content": "Write a python function that finds the nth fibonacci number"}
   ],
   task="coding"
)

print(response1.choices[0].message.content)

def fib(n):
    """
    Return the nth Fibonacci number with F(0)=0, F(1)=1 (0-indexed).
    Uses fast doubling (O(log n)).
    """
    if n < 0:
        raise ValueError("n must be >= 0")

    def _fib(k):
        if k == 0:
            return 0, 1
        a, b = _fib(k // 2)
        c = a * (2 * b - a)
        d = a * a + b * b
        if k % 2 == 0:
            return c, d
        else:
            return d, c + d

    return _fib(n)[0]


In [9]:
response1._deimos_metadata

{'router_used': 'auto-router',
 'selected_model': 'openai/gpt-5',
 'original_model_field': 'openai/gpt-5',
 'available_models': [],
 'explain': [{'rule_type': 'AutoTaskRule',
   'rule_name': 'auto-task-rule',
   'rule_trigger': 'writing code',
   'decision': 'openai/gpt-5'}]}

In [10]:
haiku = chat.completions.create(
   model="deimos/auto-router",
   messages=[
       {"role": "user", "content": "Write a short japanese poem about mars, and follow the syllable count 5, 7, 5"}
   ],
)

print(haiku.choices[0].message.content)

あかいほし
かせいはゆうひ
ゆめのつゆ


In [11]:
haiku._deimos_metadata

{'router_used': 'auto-router',
 'selected_model': 'openai/gpt-5-nano',
 'original_model_field': 'openai/gpt-5-nano',
 'available_models': [],
 'explain': [{'rule_type': 'AutoTaskRule',
   'rule_name': 'auto-task-rule',
   'rule_trigger': 'haiku composition',
   'decision': 'openai/gpt-5-nano'}]}

## Code / Not code Routing

The `CodeRule` is a very simple rule that determines whether a prompt contains code and routes based on that.

In [12]:
code_or_not = CodeRule(
    name = "code-or-not-code",
    code = "openai/gpt-5",
    not_code = "openai/gpt-4o"
)

code_no_code_router = Router(
    name = "code-nocode-router",
    rules = [code_or_not]
)



In [13]:
prompt = """
Debug this:
```
def multiply(x, y):
    return x**y
```
"""


might_be_code_1 = chat.completions.create(
   model="deimos/code-nocode-router",
   messages=[
       {"role": "user", "content": prompt}
   ],
)

print(might_be_code_1.choices[0].message.content)

You’re using exponentiation (**) instead of multiplication (*).

Fixed code:
```
def multiply(x, y):
    return x * y
```

Quick tests:
- multiply(2, 3) -> 6
- multiply(-4, 5) -> -20
- multiply(2.5, 4) -> 10.0

If you actually wanted exponentiation, rename the function or keep x ** y.


In [14]:
might_be_code_1._deimos_metadata

{'router_used': 'code-nocode-router',
 'selected_model': 'openai/gpt-5',
 'original_model_field': 'openai/gpt-5',
 'available_models': [],
 'explain': [{'rule_type': 'CodeRule',
   'rule_name': 'code-or-not-code',
   'rule_trigger': 'code_detected',
   'decision': 'openai/gpt-5'}]}

In [15]:
prompt = """
What is the smallest, ugliest moon of Mars?
"""


might_be_code_2 = chat.completions.create(
   model="deimos/code-nocode-router",
   messages=[
       {"role": "user", "content": prompt}
   ],
)

print(might_be_code_2.choices[0].message.content)

Mars has two moons, Phobos and Deimos. Of the two, Deimos is the smaller and is often described as the less interesting or less aesthetically appealing. With a diameter of about 12.4 kilometers (7.7 miles), it is irregularly shaped and appears more like an asteroid than a traditional spherical moon. Its surface is covered in dust and loose rocks, and it has fewer large craters compared to Phobos. While beauty is subjective, Deimos is often considered the "uglier" moon due to its irregular shape and less dramatic features compared to Phobos.


In [16]:
might_be_code_2._deimos_metadata

{'router_used': 'code-nocode-router',
 'selected_model': 'openai/gpt-4o',
 'original_model_field': 'openai/gpt-4o',
 'available_models': [],
 'explain': [{'rule_type': 'CodeRule',
   'rule_name': 'code-or-not-code',
   'rule_trigger': 'no_code_detected',
   'decision': 'openai/gpt-4o'}]}

## Code Language Routing

To route based on programming language, use a CodeLanguage rule.

This rule uses regex to classify among several popular languages, and then falls back to a small language model call to determine language.

In [17]:
code_lang_rule = CodeLanguageRule(
    name = "code-lang-rule",
    language_mappings = {
        "python" : "openai/gpt-3.5-turbo",
        "sql" : "openai/gpt-5-mini",
    }
)

Router(
    name = "code-lang-router",
    rules = [code_lang_rule]
)


Router(name='code-lang-router', models=[])

In [18]:
prompt = """
What does this function do?
```
def _idk(x):
    if n < 0:
        raise ValueError("n must be non-negative")
    a, b = 0, 1  # (F(m), F(m+1))
    for bit in bin(n)[2:]:
        c = a * (2 * b - a)     # F(2m)
        d = a * a + b * b       # F(2m+1)
        if bit == '0':
            a, b = c, d
        else:
            a, b = d, c + d
    return a
```
"""

what_it_do = chat.completions.create(
   model="deimos/code-lang-router",
   messages=[
       {"role": "user", "content": prompt}
   ],
)

print(what_it_do.choices[0].message.content)

This function computes the nth Fibonacci number using a technique known as binary exponentiation. It iterates through the binary representation of the input `n` and computes the appropriate Fibonacci numbers based on the current bit in the binary representation. The function then returns the nth Fibonacci number. It raises a `ValueError` if the input `n` is negative.


In [19]:
what_it_do._deimos_metadata

{'router_used': 'code-lang-router',
 'selected_model': 'openai/gpt-3.5-turbo',
 'original_model_field': 'openai/gpt-3.5-turbo',
 'available_models': [],
 'explain': [{'rule_type': 'CodeLanguageRule',
   'rule_name': 'code-lang-rule',
   'rule_trigger': 'python',
   'decision': 'openai/gpt-3.5-turbo'}]}

In [20]:
prompt = """
What does code do?
```
SELECT *
FROM people
WHERE name = "john"
```
"""

what_it_do = chat.completions.create(
   model="deimos/code-lang-router",
   messages=[
       {"role": "user", "content": prompt}
   ],
)

print(what_it_do.choices[0].message.content)

This is an SQL query. In plain terms it asks the database to:

- Read from the table named people
- Return all columns (*) for every row whose name equals "john"

Notes and caveats:
- SQL string literals are normally written with single quotes: WHERE name = 'john'. Some databases (e.g. MySQL in default mode) accept double quotes as string delimiters, but others (e.g. PostgreSQL) treat double quotes as identifiers and will error or behave differently.
- Case sensitivity depends on the database and column collation — many setups will match "John" and "john" the same, some will not.
- SELECT * returns every column; for clarity and performance it’s better to list only the columns you need.
- For variable input, avoid building queries by concatenation (risk of SQL injection). Use parameterized queries/prepared statements.
- If you expect many rows, consider adding LIMIT or an index on name for performance.


In [21]:
what_it_do._deimos_metadata

{'router_used': 'code-lang-router',
 'selected_model': 'openai/gpt-5-mini',
 'original_model_field': 'openai/gpt-5-mini',
 'available_models': [],
 'explain': [{'rule_type': 'CodeLanguageRule',
   'rule_name': 'code-lang-rule',
   'rule_trigger': 'sql',
   'decision': 'openai/gpt-5-mini'}]}

## NaturalLanguageRule

To route based on the language of the request (English, French, Spanish, etc), use a NaturalLanguageRule. This calls a small language model to detect the language. Specify language:model mapping using the two-letter ISO language code (`EN`, `FR`, `ES`).

In [23]:
nat_lang_rule = NaturalLanguageRule(
    name = "nat-lang-rule",
    language_mappings = {
        "EN" : "openai/gpt-3.5-turbo",
        "ES" : "openai/gpt-5-mini",
    }
)

Router(
    name = "nat-lang-router",
    rules = [nat_lang_rule]
)


Router(name='nat-lang-router', models=[])

In [24]:
prompt = """
Tell me a joke about language.
"""

lang_joke = chat.completions.create(
   model="deimos/nat-lang-router",
   messages=[
       {"role": "user", "content": prompt}
   ],
)

print(lang_joke.choices[0].message.content)

Why did the grammar teacher go to the beach?

To catch some clauses!


In [25]:
lang_joke._deimos_metadata

{'router_used': 'nat-lang-router',
 'selected_model': 'openai/gpt-3.5-turbo',
 'original_model_field': 'openai/gpt-3.5-turbo',
 'available_models': [],
 'explain': [{'rule_type': 'NaturalLanguageRule',
   'rule_name': 'nat-lang-rule',
   'rule_trigger': 'None',
   'decision': 'openai/gpt-3.5-turbo'}]}

In [26]:
prompt = """
Cuéntame un chiste sobre el lenguaje.
"""

lang_joke = chat.completions.create(
   model="deimos/nat-lang-router",
   messages=[
       {"role": "user", "content": prompt}
   ],
)

print(lang_joke.choices[0].message.content)

¿Sabes por qué el verbo fue al psicólogo? — Porque tenía problemas de concordancia.


In [27]:
lang_joke._deimos_metadata

{'router_used': 'nat-lang-router',
 'selected_model': 'openai/gpt-5-mini',
 'original_model_field': 'openai/gpt-5-mini',
 'available_models': [],
 'explain': [{'rule_type': 'NaturalLanguageRule',
   'rule_name': 'nat-lang-rule',
   'rule_trigger': 'None',
   'decision': 'openai/gpt-5-mini'}]}